In [1]:
import openai
import pandas as pd
import math
import re
from tqdm import tqdm
import numpy as np

In [2]:
with open("Key_for_CC_project_skim.txt", "r") as file:
    api_key = file.read().strip()
# OpenAI API 키 설정
openai.api_key = api_key

In [3]:
## 평균 응답수
total_responses = 5149
total_technologies = 241
average_responses_per_technology = total_responses / total_technologies
print(f"Average responses per technology: {average_responses_per_technology}")
# 평균 응답 수를 올림하여 총 전문가 수 설정
total_experts = math.ceil(average_responses_per_technology)
print(f"총 전문가 수: {total_experts}")

Average responses per technology: 21.365145228215766
총 전문가 수: 22


In [17]:
## 전문가 수 결정
import pandas as pd

# 소속기관 구성 비율
industry_ratio = 28 / 100
academic_ratio = 31 / 100
research_ratio = 39 / 100
other_ratio = 2 / 100

# 연구 경력 비율
experience_ratios = {
    "5년 미만": 3.7 / 100,
    "5-10년": 7.5 / 100,
    "10-15년": 11.7 / 100,
    "15-20년": 14.9 / 100,
    "20년 이상": 62.3 / 100
}

# 전문가 총 수 (예시: 100명으로 설정)
# total_experts = 100

# 각 소속기관별 전문가 수
num_industry = int(total_experts * industry_ratio)
num_academic = int(total_experts * academic_ratio)
num_research = int(total_experts * research_ratio)
num_other = int(total_experts * other_ratio)

# 소속기관 및 연구 경력에 따른 전문가 프로필 생성 함수
def generate_experts(num_experts, affiliation):
    experts = []
    remaining_experts = num_experts  # 나중에 추가할 전문가 수를 추적하기 위한 변수
    
    for experience, ratio in experience_ratios.items():
        num_experience = round(num_experts * ratio)
        if num_experience > remaining_experts:
            num_experience = remaining_experts
        for i in range(num_experience):
            experts.append({
                "name": f"Expert_{affiliation}_{len(experts) + 1}",
                "position": "연구원" if affiliation == "연구계" else "교수" if affiliation == "학계" else "엔지니어" if affiliation == "산업계" else "기타",
                "organization": f"{affiliation} 기관",
                "experience": experience,
                "specialty": f"{affiliation} 분야 전문가"
            })
        remaining_experts -= num_experience

    # 남은 전문가 수를 랜덤한 경력 레벨에 추가
    while remaining_experts > 0:
        random_experience = random.choice(experience_levels)
        experts.append({
            "name": f"Expert_{affiliation}_{len(experts) + 1}",
            "position": "연구원" if affiliation == "연구계" else "교수" if affiliation == "학계" else "엔지니어" if affiliation == "산업계" else "기타",
            "organization": f"{affiliation} 기관",
            "experience": random_experience,
            "specialty": f"{affiliation} 분야 전문가"
        })
        remaining_experts -= 1

    return experts

# 전문가 프로필 리스트 생성
experts = []
experts.extend(generate_experts(num_industry, "산업계"))
experts.extend(generate_experts(num_academic, "학계"))
experts.extend(generate_experts(num_research, "연구계"))
experts.extend(generate_experts(num_other, "기타"))

# 데이터프레임으로 변환
df_experts = pd.DataFrame(experts)
df_experts


,name,position,organization,experience,specialty
0,Expert_산업계_1,엔지니어,산업계 기관,10-15년,산업계 분야 전문가
1,Expert_산업계_2,엔지니어,산업계 기관,15-20년,산업계 분야 전문가
2,Expert_산업계_3,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가
3,Expert_산업계_4,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가
4,Expert_산업계_5,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가
5,Expert_산업계_6,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가
6,Expert_학계_1,교수,학계 기관,10-15년,학계 분야 전문가
7,Expert_학계_2,교수,학계 기관,15-20년,학계 분야 전문가
8,Expert_학계_3,교수,학계 기관,20년 이상,학계 분야 전문가
9,Expert_학계_4,교수,학계 기관,20년 이상,학계 분야 전문가


In [15]:
generate_experts(num_industry, "산업계")

[{'name': 'Expert_1',
  'position': '엔지니어',
  'organization': '산업계 기관',
  'experience': '20년 이상',
  'specialty': '산업계 분야 전문가'},
 {'name': 'Expert_2',
  'position': '엔지니어',
  'organization': '산업계 기관',
  'experience': '20년 이상',
  'specialty': '산업계 분야 전문가'},
 {'name': 'Expert_3',
  'position': '엔지니어',
  'organization': '산업계 기관',
  'experience': '20년 이상',
  'specialty': '산업계 분야 전문가'}]

In [44]:
# 연구 경력 비율
experience_ratios = {
    "5년 미만": 3.7 / 100,
    "5-10년": 7.5 / 100,
    "10-15년": 11.7 / 100,
    "15-20년": 14.9 / 100,
    "20년 이상": 62.3 / 100
}

# 소속기관 및 연구 경력에 따른 전문가 프로필 생성 함수
def generate_experts(num_experts, affiliation):
    experts = []
    remaining_experts = num_experts
    experience_levels = list(experience_ratios.keys())
    
    for experience, ratio in experience_ratios.items():
        num_experience = round(num_experts * ratio)
        if num_experience > remaining_experts:
            num_experience = remaining_experts
        for i in range(num_experience):
            expert = {
                "name": f"Expert_{affiliation}_{len(experts) + 1}",
                "position": "연구원" if affiliation == "연구계" else "교수" if affiliation == "학계" else "엔지니어" if affiliation == "산업계" else "기타",
                "organization": f"{affiliation} 기관",
                "experience": experience,
                "specialty": f"{affiliation} 분야 전문가"
            }
            # 전문가 프로필의 자세한 설명 생성
            expert["persona"] = generate_persona(expert)
            experts.append(expert)
        remaining_experts -= num_experience

    # 남은 전문가 수를 랜덤한 경력 레벨에 추가
    while remaining_experts > 0:
        random_experience = random.choice(experience_levels)
        expert = {
            "name": f"Expert_{affiliation}_{len(experts) + 1}",
            "position": "연구원" if affiliation == "연구계" else "교수" if affiliation == "학계" else "엔지니어" if affiliation == "산업계" else "기타",
            "organization": f"{affiliation} 기관",
            "experience": random_experience,
            "specialty": f"{affiliation} 분야 전문가"
        }
        expert["persona"] = generate_persona(expert)
        experts.append(expert)
        remaining_experts -= 1

    return experts

# 함수: 전문가의 자세한 persona 생성
def generate_persona(expert):
    prompt = (f"Generate a detailed persona for a professional in the {expert['specialty']}. "
              f"The persona should include a brief background, specific skills, key achievements, "
              f"and the professional's approach to their field.\n\n"
              f"Name: {expert['name']}\n"
              f"Position: {expert['position']}\n"
              f"Organization: {expert['organization']}\n"
              f"Experience: {expert['experience']}\n"
              f"Specialty: {expert['specialty']}\n\n"
              "Persona Description:")
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI assistant that helps generate detailed professional personas."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    persona_description = response.choices[0].message.content.strip()

    final_persona = (f"{persona_description}\n\n"
                     f"Experience: {expert['experience']}\n"
                     f"Organization: {expert['organization']}")
    
    return final_persona

# 총 응답 수와 총 기술 수
total_responses = 5149
total_technologies = 241

# 평균 응답 수 계산
average_responses_per_technology = total_responses / total_technologies

# 평균 응답 수를 올림하여 총 전문가 수 설정
total_experts = math.ceil(average_responses_per_technology)

# 소속기관 구성 비율
affiliation_ratios = {
    "산업계": 28 / 100,
    "학계": 31 / 100,
    "연구계": 39 / 100,
    "기타": 2 / 100
}

# 전문가 프로필 리스트 생성
experts = []
for affiliation, ratio in affiliation_ratios.items():
    num_experts = int(total_experts * ratio)
    experts.extend(generate_experts(num_experts, affiliation))

# 데이터프레임으로 변환
df_experts = pd.DataFrame(experts)
df_experts

,name,position,organization,experience,specialty,persona
0,Expert_산업계_1,엔지니어,산업계 기관,10-15년,산업계 분야 전문가,### Professional Persona: Expert_산업계_1 \n\n**N...
1,Expert_산업계_2,엔지니어,산업계 기관,15-20년,산업계 분야 전문가,### Persona: Expert_산업계_2\n\n**Name:** 박지현 (Pa...
2,Expert_산업계_3,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가,### Professional Persona: Expert_산업계_3\n\n**Na...
3,Expert_산업계_4,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가,### Persona: Expert_산업계_4\n\n**Name:** Expert_...
4,Expert_산업계_5,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가,### Persona: Expert_산업계_5\n\n**Name:** 김영수 (Ki...
5,Expert_산업계_6,엔지니어,산업계 기관,20년 이상,산업계 분야 전문가,**Name:** Expert_산업계_6 \n**Position:** 엔지니어 ...
6,Expert_학계_1,교수,학계 기관,10-15년,학계 분야 전문가,### Persona: Expert_학계_1\n\n**Name:** Dr. Jiso...
7,Expert_학계_2,교수,학계 기관,15-20년,학계 분야 전문가,### Persona Description\n\n**Name:** Dr. Min-J...
8,Expert_학계_3,교수,학계 기관,20년 이상,학계 분야 전문가,### Persona Profile: Expert_학계_3\n\n**Name:** ...
9,Expert_학계_4,교수,학계 기관,20년 이상,학계 분야 전문가,### Persona: Expert_학계_4\n\n**Name:** Dr. Min-...


In [1]:
df_experts['persona'].iloc[0]

NameError: name 'df_experts' is not defined

In [45]:
# 설문 질문 리스트
questions = [
    "금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까? (1: 전혀 실현되지 않음, 5: 완전히 실현됨)",
    "금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까? (1: 1-2년, 5: 12년 이상)",
    "금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까? (1: 1-2년, 5: 12년 이상)",
    "국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까? (1: 격차 없음, 5: 7년 이상)"
]

# 함수: 각 질문에 대해 전문가의 응답 생성
def generate_survey_response(expert, question):
    prompt = (f"Based on the following professional profile, provide a response to the question.\n\n"
              f"Persona: {expert['persona']}\n\n"
              f"Question: {question}\n\n"
              "Provide a response on a scale of 1 to 5, with a brief explanation if necessary.")
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI assistant that helps generate expert opinions on technical questions."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=0.7
    )

    # 결과에서 점수와 설명을 추출
    answer = response.choices[0].message.content.strip()
    
    return answer

# 각 프로필에 대해 질문에 대한 답변 생성
survey_responses = []
for _, expert in df_experts.iterrows():
    for question in questions:
        answer = generate_survey_response(expert, question)
        survey_responses.append({
            "name": expert["name"],
            "question": question,
            "answer": answer
        })

# 응답 데이터프레임으로 변환
df_survey_responses = pd.DataFrame(survey_responses)
df_survey_responses


,name,question,answer
0,Expert_산업계_1,금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까? (1: 전혀 실현되...,저는 금속 3D 프린팅 기술의 현재 실현 여부를 4로 평가합니다. 금속 3D 프린팅...
1,Expert_산업계_1,금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까? (1...,"제 개인적인 의견으로는, 금속 3D 프린팅 기술의 완전한 실현까지는 4점, 즉 약 ..."
2,Expert_산업계_1,금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까? (1: 1...,**Rating: 4 (5-10년)**\n\n금속 3D 프린팅 기술은 이미 여러 산...
3,Expert_산업계_1,국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까? (1: 격...,Based on my experience and observations in the...
4,Expert_산업계_2,금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까? (1: 전혀 실현되...,"금속 3D 프린팅 기술의 현재 실현 여부를 평가하자면, 저는 4점을 주겠습니다. \..."
...,...,...,...
75,Expert_연구계_7,국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까? (1: 격...,Based on my experience and observations in the...
76,Expert_연구계_8,금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까? (1: 전혀 실현되...,"금속 3D 프린팅 기술의 현재 실현 여부를 평가한다면, 저는 4로 평가하겠습니다.\..."
77,Expert_연구계_8,금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까? (1...,Based on my expertise and understanding of tec...
78,Expert_연구계_8,금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까? (1: 1...,Based on my expertise and understanding of the...


In [47]:
df_survey_responses['answer'].iloc[0]

'저는 금속 3D 프린팅 기술의 현재 실현 여부를 4로 평가합니다. 금속 3D 프린팅 기술은 최근 몇 년간 상당한 발전을 이루었으며, 특히 항공우주, 자동차 및 의료 분야에서 실제로 적용되고 있습니다. 많은 기업들이 이 기술을 통해 맞춤형 부품 생산과 경량화 설계를 구현하고 있으며, 이는 생산 효율성과 비용 절감에 기여하고 있습니다.\n\n그러'

In [8]:
# 설문 질문 리스트
questions = [
    "금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까?",
    "금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까?",
    "금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까?",
    "금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? (1점: 매우 낮음, 5점: 매우 높음)",
    "금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? (1점: 매우 낮음, 5점: 매우 높음)",
    "국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까?"
]

In [37]:
response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI assistant that helps generate detailed professional personas."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )

In [36]:
prompt = (f"Generate a detailed persona for a professional in the {expert['specialty']}. "
              f"The persona should include a brief background, specific skills, key achievements, "
              f"and the professional's approach to their field.\n\n"
              f"Name: {expert['name']}\n"
              f"Position: {expert['position']}\n"
              f"Organization: {expert['organization']}\n"
              f"Experience: {expert['experience']}\n"
              f"Specialty: {expert['specialty']}\n\n"
              "Persona Description:")

In [35]:
expert = {
                "name": f"Expert_{affiliation}_{len(experts) + 1}",
                "position": "연구원" if affiliation == "연구계" else "교수" if affiliation == "학계" else "엔지니어" if affiliation == "산업계" else "기타",
                "organization": f"{affiliation} 기관",
                "experience": experience,
                "specialty": f"{affiliation} 분야 전문가"
            }

In [31]:
experience_ratios.items()

dict_items([('5년 미만', 0.037000000000000005), ('5-10년', 0.075), ('10-15년', 0.11699999999999999), ('15-20년', 0.149), ('20년 이상', 0.623)])

In [33]:
experience = '5년 미만'

In [34]:
ratio = 0.037000000000000005

In [38]:
response

ChatCompletion(id='chatcmpl-9u9Otnq9OXunpixZaNpDJ1oJ4xBak', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="### Persona: Expert_산업계_1\n\n**Name:** 박지훈 (Park Ji-hoon)  \n**Position:** 엔지니어 (Engineer)  \n**Organization:** 산업계 기관 (Industrial Sector Organization)  \n**Experience:** 5년 미만 (Less than 5 years)  \n**Specialty:** 산업계 분야 전문가 (Industrial Sector Expert)  \n\n---\n\n#### **Background:**\n박지훈 is a dynamic and innovative engineer with a Bachelor's degree in Mechanical Engineering from Seoul National University. Since graduating in 2020, he has quickly established himself in the industrial sector, working for a prominent organization dedicated to advancing industrial technologies and practices. Ji-hoon grew up in a family of engineers, which fostered his passion", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723169531, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, 

In [39]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

In [40]:
response['choices'][0]['message']['content'].strip()

TypeError: 'ChatCompletion' object is not subscriptable

In [41]:
response.choices[0].message['content']

TypeError: 'ChatCompletionMessage' object is not subscriptable

In [42]:
response.choices[0]

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="### Persona: Expert_산업계_1\n\n**Name:** 박지훈 (Park Ji-hoon)  \n**Position:** 엔지니어 (Engineer)  \n**Organization:** 산업계 기관 (Industrial Sector Organization)  \n**Experience:** 5년 미만 (Less than 5 years)  \n**Specialty:** 산업계 분야 전문가 (Industrial Sector Expert)  \n\n---\n\n#### **Background:**\n박지훈 is a dynamic and innovative engineer with a Bachelor's degree in Mechanical Engineering from Seoul National University. Since graduating in 2020, he has quickly established himself in the industrial sector, working for a prominent organization dedicated to advancing industrial technologies and practices. Ji-hoon grew up in a family of engineers, which fostered his passion", refusal=None, role='assistant', function_call=None, tool_calls=None))

In [43]:
response.choices[0].message.content.strip()

"### Persona: Expert_산업계_1\n\n**Name:** 박지훈 (Park Ji-hoon)  \n**Position:** 엔지니어 (Engineer)  \n**Organization:** 산업계 기관 (Industrial Sector Organization)  \n**Experience:** 5년 미만 (Less than 5 years)  \n**Specialty:** 산업계 분야 전문가 (Industrial Sector Expert)  \n\n---\n\n#### **Background:**\n박지훈 is a dynamic and innovative engineer with a Bachelor's degree in Mechanical Engineering from Seoul National University. Since graduating in 2020, he has quickly established himself in the industrial sector, working for a prominent organization dedicated to advancing industrial technologies and practices. Ji-hoon grew up in a family of engineers, which fostered his passion"

In [48]:
### 질문 재생성
# 초기 설문 질문 리스트 및 관련 응답 형식
questions = [
    {
        "question": "금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까?",
        "options": {
            1: "전혀 실현되지 않음",
            2: "거의 실현되지 않음",
            3: "부분적으로 실현됨",
            4: "대부분 실현됨",
            5: "완전히 실현됨"
        }
    },
    {
        "question": "금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까?",
        "options": {
            1: "1-2년",
            2: "3-5년",
            3: "6-8년",
            4: "9-11년",
            5: "12년 이상"
        }
    },
    {
        "question": "금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까?",
        "options": {
            1: "1-2년",
            2: "3-5년",
            3: "6-8년",
            4: "9-11년",
            5: "12년 이상"
        }
    },
    {
        "question": "금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까?",
        "subquestions": ["고비용", "품질 관리의 어려움", "기술 표준화의 부족"],
        "options": {
            1: "매우 낮음",
            2: "낮음",
            3: "보통",
            4: "높음",
            5: "매우 높음"
        }
    },
    {
        "question": "금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까?",
        "subquestions": ["기술 혁신", "비용 절감 전략", "표준화 노력"],
        "options": {
            1: "매우 낮음",
            2: "낮음",
            3: "보통",
            4: "높음",
            5: "매우 높음"
        }
    },
    {
        "question": "국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까?",
        "options": {
            1: "격차 없음",
            2: "1-2년",
            3: "3-4년",
            4: "5-6년",
            5: "7년 이상"
        }
    },
    {
        "question": "금속 3D 프린팅 기술이 현재 어떤 산업 분야에서 가장 유망하다고 생각하십니까?",
        "response_type": "open-ended"
    },
    {
        "question": "금속 3D 프린팅 기술의 상용화를 촉진하기 위해 가장 필요한 정책적 지원은 무엇이라고 생각하십니까?",
        "response_type": "open-ended"
    }
]


In [61]:
# 함수: 각 질문에 대해 전문가의 응답 생성
def generate_survey_response(expert, question):
    if "subquestions" in question:
        responses = []
        for subquestion in question["subquestions"]:
            prompt = (f"Based on the following professional profile, select the most appropriate response to the question.\n\n"
                      f"Persona: {expert['persona']}\n\n"
                      f"Question: {question['question']} - {subquestion}\n\n"
                      "Choose from the following options:\n" +
                      "\n".join([f"{k}: {v}" for k, v in question['options'].items()]) +
                      "\nProvide the option number (1-5) as the answer with a brief justification.")
            
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are an AI assistant that helps generate expert opinions on technical questions."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=150,
                temperature=0.7
            )

            answer = response.choices[0].message.content.strip()
            responses.append({
                "subquestion": subquestion,
                "answer": answer
            })
        return responses
    elif question.get("response_type") == "open-ended":
        prompt = (f"Based on the following professional profile, provide a detailed response to the open-ended question.\n\n"
                  f"Persona: {expert['persona']}\n\n"
                  f"Question: {question['question']}\n\n"
                  "Provide a detailed answer to the question.")
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an AI assistant that helps generate expert opinions on technical questions."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7
        )

        answer = response.choices[0].message.content.strip()
        return answer
    else:
        prompt = (f"Based on the following professional profile, select the most appropriate response to the question.\n\n"
                  f"Persona: {expert['persona']}\n\n"
                  f"Question: {question['question']}\n\n"
                  "Choose from the following options:\n" +
                  "\n".join([f"{k}: {v}" for k, v in question['options'].items()]) +
                  "\nProvide the option number (1-5) as the answer with a brief justification.")
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an AI assistant that helps generate expert opinions on technical questions."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7
        )

        answer = response.choices[0].message.content.strip()
        return answer

# 각 프로필에 대해 질문에 대한 답변 생성
survey_responses = []
for _, expert in df_experts.iterrows():
    for question in questions:
        answer = generate_survey_response(expert, question)
        if isinstance(answer, list):
            for sub_ans in answer:
                survey_responses.append({
                    "name": expert["name"],
                    "question": f"{question['question']} - {sub_ans['subquestion']}",
                    "answer": sub_ans["answer"]
                })
        else:
            survey_responses.append({
                "name": expert["name"],
                "question": question["question"],
                "answer": answer
            })

# 응답 데이터프레임으로 변환
df_survey_responses = pd.DataFrame(survey_responses)
df_survey_responses


,name,question,answer
0,Expert_산업계_1,금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까?,"4: 대부분 실현됨\n\nCurrently, metal 3D printing tec..."
1,Expert_산업계_1,금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까?,2: 3-5년\n\nJustification: 금속 3D 프린팅 기술은 현재도 상당...
2,Expert_산업계_1,금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까?,2: 3-5년\n\nJustification: 금속 3D 프린팅 기술은 이미 상당한...
3,Expert_산업계_1,금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? -...,4: 높음\n\nJustification: 금속 3D 프린팅 기술은 고비용이 주요 ...
4,Expert_산업계_1,금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? -...,4: 높음\n\nJustification: 금속 3D 프린팅 기술에서 품질 관리의 ...
...,...,...,...
235,Expert_연구계_8,금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? ...,4: 높음\n\nJustification: Cost reduction strateg...
236,Expert_연구계_8,금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? ...,4: 높음\n\nJustification: 표준화 노력은 금속 3D 프린팅 기술의 ...
237,Expert_연구계_8,국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까?,**Answer: 3**\n\n**Justification:** Based on m...
238,Expert_연구계_8,금속 3D 프린팅 기술이 현재 어떤 산업 분야에서 가장 유망하다고 생각하십니까?,"금속 3D 프린팅 기술은 최근 몇 년 동안 그 발전 속도가 가속화되었으며, 여러 산..."


In [51]:
questions

[{'question': '금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까?',
  'options': {1: '전혀 실현되지 않음',
   2: '거의 실현되지 않음',
   3: '부분적으로 실현됨',
   4: '대부분 실현됨',
   5: '완전히 실현됨'}},
 {'question': '금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까?',
  'options': {1: '1-2년', 2: '3-5년', 3: '6-8년', 4: '9-11년', 5: '12년 이상'}},
 {'question': '금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까?',
  'options': {1: '1-2년', 2: '3-5년', 3: '6-8년', 4: '9-11년', 5: '12년 이상'}},
 {'question': '금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까?',
  'subquestions': ['고비용', '품질 관리의 어려움', '기술 표준화의 부족'],
  'options': {1: '매우 낮음', 2: '낮음', 3: '보통', 4: '높음', 5: '매우 높음'}},
 {'question': '금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까?',
  'subquestions': ['기술 혁신', '비용 절감 전략', '표준화 노력'],
  'options': {1: '매우 낮음', 2: '낮음', 3: '보통', 4: '높음', 5: '매우 높음'}},
 {'question': '국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까?',
  'options': {1: '격차 없음', 2: '1-2년', 3: '3-4년', 4: '5-6년', 5: '7년 이상'}},
 {'question': '금속 3D 프린팅 기술이 현재 어떤 산업 분야에서 가장 유망하다고 생각하십니까?',
  'response_type

In [62]:
df_survey_responses['answer'].iloc[3]

'4: 높음\n\nJustification: 금속 3D 프린팅 기술은 고비용이 주요 장애 요인으로 작용하고 있습니다. 초기 투자 비용, 재료 비용 및 장비 유지 관리 비용이 높기 때문에 많은 기업들이 이 기술을 채택하는 데 어려움을 겪습니다. 이는 특히 중소기업이나 자본이 제한된 기업에게 큰 장벽이 됩니다. 따라서, 금속 3D 프린팅 기술의 상용화와 확산을 저해하는 중요한 요소로 평가할 수 있습니다.'

In [63]:
df_survey_responses['answer'].iloc[4]

'4: 높음\n\nJustification: 금속 3D 프린팅 기술에서 품질 관리의 어려움은 중요한 장애 요인 중 하나입니다. 이 기술은 복잡한 제조 공정을 포함하며, 재료의 특성과 프린팅 조건에 따라 결과물이 크게 달라질 수 있습니다. 따라서 일관된 품질을 유지하는 것은 매우 도전적이며, 이는 생산 과정에서의 불량률 증가와 고객의 신뢰도 저하로 이어질 수 있습니다. 따라서 품질 관리의 어려움은 금속 3D 프린팅 기술의 실현을 방해하는 주요 요인으로 평가할 수 있습니다.'

In [64]:
df_survey_responses['answer'].iloc[5]

'4: 높음\n\nJustification: 기술 표준화의 부족은 금속 3D 프린팅 기술의 실현에 큰 장애 요소로 작용할 수 있습니다. 표준화가 이루어지지 않으면, 다양한 장비와 소재 간의 호환성 문제, 품질 관리의 어려움, 그리고 생산 과정에서의 일관성 부족 등으로 인해 산업 전반에 걸쳐 효율성이 떨어질 수 있습니다. 이러한 문제는 특히 제조업에서의 채택과 확산을 저해할 수 있으므로, 이 문제의 심각성을 높게 평가합니다.'

In [65]:
df_survey_responses['answer'].iloc[6]

'5: 매우 높음\n\nJustification: 기술 혁신은 금속 3D 프린팅 기술의 발전에 있어 핵심적인 요소입니다. 이 기술은 전통적인 제조 방식에 비해 설계의 자유도를 증가시키고, 자원 절약과 생산 효율성을 높이며, 복잡한 구조물의 제작을 가능하게 합니다. 특히, 지속 가능한 제조와 맞물려 환경 영향을 최소화할 수 있는 잠재력이 크기 때문에, 기술 혁신의 중요성은 매우 높다고 평가합니다.'

In [66]:
df_survey_responses['answer'].iloc[7]

'4: 높음\n\nJustification: 금속 3D 프린팅 기술의 실현을 촉진하는 데 있어 비용 절감 전략은 매우 중요한 요소입니다. 이 기술은 초기 투자 비용이 높지만, 장기적으로는 생산 과정에서의 재료 낭비 감소, 맞춤형 생산 가능성 및 재고 관리 효율성을 통해 전체 운영 비용을 절감할 수 있습니다. 따라서 효율적인 비용 관리와 절감 전략을 통해 금속 3D 프린팅의 채택을 가속화할 수 있습니다.'

In [67]:
df_survey_responses['answer'].iloc[9]

'4: 5-6년\n\nJustification: Based on my experience in the industrial sector, I believe that the gap between domestic and international metal 3D printing technologies is around 5-6 years. While South Korea has made significant strides in this field, particularly in research and development, countries with more established manufacturing ecosystems and investment in advanced technologies, such as the United States and Germany, continue to lead in practical applications and scalability of metal 3D printing. This gap reflects differences in technological adoption, infrastructure, and investment in innovation.'

In [71]:
df_survey_responses['answer'].iloc[19]

'4: 높음\n\n금속 3D 프린팅 기술은 기존 제조 공정에 비해 초기 투자 비용이 높은 경향이 있지만, 장기적으로는 생산 비용 절감과 자원 효율성을 높일 수 있는 잠재력을 가지고 있습니다. 특히, 복잡한 부품을 한 번에 제작할 수 있어 후처리 비용을 줄이고, 재료 낭비를 최소화하는 데 기여할 수 있습니다. 따라서, 비용 절감 전략이 금속 3D 프린팅 기술의 실현을 촉진하는 중요한 요인으로 작용한다고 평가합니다.'

In [72]:
# 객관식 질문 통계 계산 함수
def calculate_statistics(df):
    statistics = {}
    
    # 객관식 질문 필터링
    objective_questions = df[~df['question'].str.contains('open-ended')]
    
    for question in objective_questions['question'].unique():
        # 해당 질문에 대한 응답만 추출
        question_responses = objective_questions[objective_questions['question'] == question]['answer']
        
        # 응답에서 숫자만 추출하여 통계 계산
        numeric_responses = question_responses.str.extract(r'(\d)').astype(int)
        stats = numeric_responses.value_counts().sort_index()
        
        # 결과 저장
        statistics[question] = stats
    
    return statistics

# 자유 형식 질문 응답 모으기 함수
def collect_open_ended_responses(df):
    responses = {}
    
    # 자유 형식 질문 필터링
    open_ended_questions = df[df['question'].str.contains('open-ended')]
    
    for question in open_ended_questions['question'].unique():
        # 해당 질문에 대한 응답만 추출
        question_responses = open_ended_questions[open_ended_questions['question'] == question]['answer'].tolist()
        
        # 결과 저장
        responses[question] = question_responses
    
    return responses

# 객관식 질문 통계 계산
objective_statistics = calculate_statistics(df_survey_responses)

# 자유 형식 질문 응답 모으기
open_ended_responses = collect_open_ended_responses(df_survey_responses)

# 결과 출력 (예시)
print("객관식 질문 통계:")
for question, stats in objective_statistics.items():
    print(f"\n{question}")
    print(stats)

print("\n자유 형식 질문 응답:")
for question, responses in open_ended_responses.items():
    print(f"\n{question}")
    for response in responses:
        print(f"- {response}")


객관식 질문 통계:

금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까?
0
3    11
4     9
Name: count, dtype: int64

금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까?
0
2    12
3     8
Name: count, dtype: int64

금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까?
0
2    15
3     4
5     1
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? - 고비용
0
4    16
5     4
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? - 품질 관리의 어려움
0
4    20
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? - 기술 표준화의 부족
0
3     1
4    18
5     1
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? - 기술 혁신
0
3     1
4    14
5     5
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? - 비용 절감 전략
0
3     2
4    18
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? - 표준화 노력
0
3     2
4    18
Name: count, dtype: int64

국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까?
0
3    18
4     1
7     1
Name: count, dtype: int64

금속 3D 프린팅 기술이 현재 어떤

In [74]:
# 객관식 질문 통계 계산 함수
def calculate_statistics(df):
    statistics = {}
    
    # 객관식 질문 필터링
    for question in questions:
        if "response_type" not in question:  # 객관식 질문만 처리
            if "subquestions" in question:  # 하위 질문이 있는 경우
                for subquestion in question["subquestions"]:
                    # 해당 하위 질문에 대한 응답만 추출
                    subquestion_full = f"{question['question']} - {subquestion}"
                    question_responses = df[df['question'] == subquestion_full]['answer']
                    
                    # 응답에서 숫자만 추출하여 통계 계산
                    numeric_responses = question_responses.str.extract(r'(\d)').astype(int)
                    stats = numeric_responses.value_counts().sort_index()
                    
                    # 결과 저장
                    statistics[subquestion_full] = stats
            else:  # 일반 객관식 질문
                question_responses = df[df['question'] == question['question']]['answer']
                
                # 응답에서 숫자만 추출하여 통계 계산
                numeric_responses = question_responses.str.extract(r'(\d)').astype(int)
                stats = numeric_responses.value_counts().sort_index()
                
                # 결과 저장
                statistics[question['question']] = stats
    
    return statistics

# 자유 형식 질문 응답 모으기 함수
def collect_open_ended_responses(df):
    responses = {}
    
    # 자유 형식 질문 필터링
    for question in questions:
        if question.get("response_type") == "open-ended":  # 자유 형식 질문만 처리
            # 해당 질문에 대한 응답만 추출
            question_responses = df[df['question'] == question['question']]['answer'].tolist()
            
            # 결과 저장
            responses[question['question']] = question_responses
    
    return responses

# 객관식 질문 통계 계산
objective_statistics = calculate_statistics(df_survey_responses)

# 자유 형식 질문 응답 모으기
open_ended_responses = collect_open_ended_responses(df_survey_responses)

# 결과 출력 (예시)
print("객관식 질문 통계:")
for question, stats in objective_statistics.items():
    print(f"\n{question}")
    print(stats)

print("\n자유 형식 질문 응답:")
for question, responses in open_ended_responses.items():
    print(f"\n{question}")
    for response in responses:
        print(f"- {response}")


객관식 질문 통계:

금속 3D 프린팅 기술의 현재 실현 여부를 어떻게 평가하십니까?
0
3    11
4     9
Name: count, dtype: int64

금속 3D 프린팅 기술이 완전히 실현되기까지 얼마나 걸릴 것으로 예상하십니까?
0
2    12
3     8
Name: count, dtype: int64

금속 3D 프린팅 기술이 상용화되기까지 얼마나 걸릴 것으로 예상하십니까?
0
2    15
3     4
5     1
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? - 고비용
0
4    16
5     4
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? - 품질 관리의 어려움
0
4    20
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 방해하는 주요 장애 요인을 어떻게 평가하십니까? - 기술 표준화의 부족
0
3     1
4    18
5     1
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? - 기술 혁신
0
3     1
4    14
5     5
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? - 비용 절감 전략
0
3     2
4    18
Name: count, dtype: int64

금속 3D 프린팅 기술의 실현을 촉진할 수 있는 주요 요인을 어떻게 평가하십니까? - 표준화 노력
0
3     2
4    18
Name: count, dtype: int64

국내와 해외의 금속 3D 프린팅 기술 격차를 얼마나 된다고 생각하십니까?
0
3    18
4     1
7     1
Name: count, dtype: int64

자유 형식 질문 응답:

금속 3D